# nagai

In [ ]:
from sympy import *
import numpy as np
import math
import random
import tools as my
import time

def SquarefreeDecomposition(f):
    if content(f) != 1:
        raise ValueError('cont(f) != 1')
    s = gcd(f, diff(f, f.gen))
    if s == 1:
        return [f]
    t = []; u = []; v = []; F = []
    t.append(quo(f, s)); u.append(quo(diff(f, f.gen), s))
    v.append(u[0] - diff(t[0]))
    i = 1
    while t[i-1] != 1:
        F.append(gcd(t[i-1], v[i-1]))
        t.append(quo(t[i-1], F[i-1]))
        u.append(quo(v[i-1], F[i-1]))
        v.append(u[i] - diff(t[i], f.gen))
        i = i + 1
    return F

def Berlekamp_Matrix(f):
    # Berlekamp行列
    
    if LC(f) != 1:
        raise ValueError('f(x) is not monic.')
    if gcd(f, diff(f, f.gen, domain=f.domain)) != 1:
        raise ValueError('f(x) is not square-free.')

    x = f.gen
    p = f.domain.characteristic()
    m = degree(f)
    g0 = 1
    B = np.zeros((m, m), dtype=Integer); B[0][0] = g0
    g1 = rem(x**p, f, domain=GF(p))
    if g1 == g1.zero:
        deg_g1 = 0
    else:
        deg_g1 = degree(g1)
    for i in range(deg_g1):
        B[1][i-1] = list(reversed(g1.all_coeffs()))[i]
    g = g1
    for i in range(2, m):
        g = rem(g*g1, f, domain=GF(p))
        if g == g.zero:
            deg_g1 = 0
        else:
            deg_g = degree(g)
        for j in range(deg_g):
            B[i][j-1] = list(reversed(g.all_coeffs()))[j]
    return B

def f_ReducingPolynomial(f, B):
    # f-簡約多項式（Berlekampアルゴリズムでの分離多項式）
    
    x = f.gen
    p = f.domain.characteristic()
    m = np.shape(B)[0]
    B = B - np.identity(m, dtype=Integer)
    for i in range(m-1):
        if sum(B[i][i:] == 0) == m-i:
            continue
        j = i
        while B[i][i] == 0:
            j = j + 1
            if B[i][j] != 0:
                B[:, i], B[:, j] = B[:, j], B[:, i]    
        recip = invert(B[i][i], p)
        B[:, i] = (B[:, i]*recip) % p
        for j in range(m):
            if j == i:
                continue
            B[:, j] = (B[:, j] - B[i][j]*B[:, i]) % p
    B = np.identity(m, dtype=Integer) - B
    
    _B = np.zeros((1, m), dtype=Integer)
    for i in range(m):
        if sum(B[i] == 0) == m:
            continue
        _B = np.append(_B, [B[i]], axis=0)
    _B = _B[1:]
    f_reducing_polynomials = []
    
    k = np.shape(_B)[0]
    for i in range(k):
        _f = Poly(0, x, domain=GF(p))
        for j in range(m):
            coeff = _B[i][j]
            _f = _f + Poly(coeff*x**j, x, domain=GF(p))  
        f_reducing_polynomials.append(_f)
    return tuple(f_reducing_polynomials)

def _BerlekampAlgorithm(f, output=True):
    # Berlekampアルゴリズム（有限体上の因数分解）
    
    if LC(f) != 1:
        raise ValueError('f(x) is not monic.')
    if gcd(f, diff(f, f.gen)) != 1:
        raise ValueError('f(x) is not square-free.')
        
    B = Berlekamp_Matrix(f)
    f_reducing_polynomials = f_ReducingPolynomial(f, B)
    
    if output:
        print('Berlekamp Matrix: ' + '\n' + str(B) + '\n')
        print('f reducing polynomials:' + '\n' + str(f_reducing_polynomials) + '\n')

        print('--------------------_Berlekamp Algorithm--------------------')
    
    # 自明なf-簡約多項式はg1へ
    g1 = np.array([], dtype=Integer); g_list = np.array([], dtype=Integer)
    for g in f_reducing_polynomials:
        if degree(g) == 0:
            g1 = np.append(g1, g)
        else:
            g_list = np.append(g_list, g)
    if output:      
        print('g1 :' + str(g1)); print('g_list :' + str(g_list))
    
    #if len(f_reducing_polynomials) == 1:
    if len(g_list) == 1:
        return f
    else:
        F = np.array([f])
    alpha = range(f.domain.characteristic())
    
    # _F ... F', F_ ... F_tilde
    for i in range(len(g_list)):
        _F = np.array([], dtype=Integer)
        for _f in F:
            F_ = np.array([], dtype=Integer)
            for a in alpha:
                gcd_fg = gcd(_f, g_list[i] - a, domain=_f.domain)
                if degree(gcd_fg) != 0:
                    F_ = np.append(F_, gcd_fg)
            h = Poly(1, x, domain=_f.domain)
            for F_elements in F_:
                h = h * F_elements
            _f = quo(_f, h, domain=_f.domain)
            if _f != f.one:
                F_ = np.append(F_, _f)
            _F = np.append(_F, F_)
            print('_F:', _F)
        if len(_F) == len(f_reducing_polynomials):
            if output:
                print('rF', _F)
                print('_rF', tuple(_F))
                print('------------------------------------------------------------')
            return tuple(_F)
        F = _F

def _Zassenhaus(f):
    if primitive(f)[0] != 1:
        raise ValueError('f(x) is not primitive.')
    if gcd(f, diff(f, f.gen, domain=f.domain)) != 1:
        raise ValueError('f(x) is not square-free.')
        
    m = degree(f)
    max_f = f.max_norm()
    B_ = sqrt(m+1) * 2**m * max_f
    B = B_ * abs(LC(f))
    x = f.gen
    
    i = 10
    while True:
        i += 3
        p = prime(i)
        if (gcd(f, diff(f, x, domain=GF(p)), domain=GF(p)) == 1) and (bool(LC(f) % p)):
            break
    fp = Poly(f * invert(LC(f), p), x, domain=GF(p))
    print('fp:', fp)
    g_list = [_BerlekampAlgorithm(fp)]
    #g_list = [my.BerlekampAlgorithm(fp)]
    d = math.ceil(log(2*B+1, p)) - 1
    h_list = my.HenselLifting_MultiFactors(p, f, g_list, d, output=False)
    s = 1
    F = []; H = h_list; _f = f
    while 2*s <= len(H):
        iter_list = itertools.combinations(H, s)
        for S in iter_list:
            h_setminus_S = list(set(H) - set(S))
            f_s = Poly(LC(_f) * np.prod(S), x, domain=GF(p**(d+1))).as_poly(domain=ZZ)
            f_h_setminus_S = Poly(LC(_f) * np.prod(h_setminus_S), x, domain=GF(p**(d+1))).as_poly(domain=ZZ)
            if f_s.l1_norm() * f_h_setminus_S.l1_norm() <= B:
                H = h_setminus_S
                F.append(primitive(f_s)[1])
                _f = primitive(f_h_setminus_S)[1]
                break
        else:
            s = s + 1
    return np.append(F, _f)  

def _Zassenhaus_Algorithm(f):
    cont_f, pp_f = primitive(f)
    sf_f_list = SquarefreeDecomposition(pp_f)
    factors_list = []
    i = 1
    for sf_f in sf_f_list:
        print(sf_f)
        result = list(_Zassenhaus(sf_f))
        for _ in range(i):
            factors_list.append(result)
        i += 1
    factors_list[0] *= cont_f
    return factors_list

x = Symbol('x')
f = ((x+1)**2*(2*x-3)**3*(x**2+x+9)).as_poly(domain=ZZ)

In [ ]:
%%time
_Zassenhaus_Algorithm(f1)

In [ ]:
factor(Poly(x**7 + 4*x**6 - 11*x**5 - 8*x**4 - 3*x**3 + 2*x**2 + 15*x, x, modulus=41), domain=GF(41))

In [ ]:
%%time
my.Zassenhaus_Algorithm(f)

In [ ]:
x = Symbol('x')
h = Poly(18*x**10 + 216*x**8 - 24*x**7 + 648*x**6 - 288*x**5 + 8*x**4 - 864*x**3 + 96*x**2 + 288, x, domain=ZZ)
#h = Poly(x**5 + 9*x**4 + 6*x**3 - 3*x**2 - 27*x - 18, x, domain=ZZ)

# main

In [1]:
from sympy import *
import numpy as np
import math
import tools as my
import random
import itertools
import time
import pickle

In [2]:
coeffs_inf, coeffs_sup = -10, 10
random.seed(2525)

In [3]:
iter_num = 1
f1_time_list = []
f2_time_list = []
f3_time_list = []
f4_time_list = []

In [4]:
def my_random_poly_ZZ(*degs):
    random.seed(2525)
    x = Symbol('x')
    f = Poly(1, x, domain=ZZ)
    for deg in list(degs):
        f *= random_poly(x, deg, -3, 3, domain=ZZ, polys=True)
    return expand(f)

In [34]:
x = Symbol('x')
_f1 = my_random_poly_ZZ(2, 1, 3, 1); # deg = 7
_f2 = my_random_poly_ZZ(2, 1, 1, 4, 5); # deg = 13
#_f3 = my_random_poly_ZZ(4, 5, 1, 6, 3, 1); # deg = 20
_f3 = my_random_poly_ZZ(1, 5, 5, 3, 4, 2); # deg = 20
_f4 = my_random_poly_ZZ(7, 13, 21, 5, 4, 5, 12, 13, 17, 6, 7); # deg = 110
#_f3 = my_random_poly_ZZ(6, 2, 4, 5, 2, 6, 3, 1, 3, 4); _f3 = quo(_f3, (x-1))# deg = 30


In [35]:
f1, f2, f3 = (_f1/content(_f1)).as_poly(domain=ZZ), (_f2/content(_f2)).as_poly(domain=ZZ), (_f3/content(_f3)).as_poly(domain=ZZ)
f4 = (_f4/content(_f4)).as_poly(domain=ZZ)
h = Poly(3*x**5 + 18*x**3 - 2*x**2 - 12, x, domain=ZZ)

In [ ]:
%%time
func_list = [factor, my._Zassenhaus_Algorithm, my.Zassenhaus_Algorithm]
for func in func_list:
    print(func)
    t1 = time.process_time()
    for i in range(iter_num):
        result = func(f1)
    t2 = time.process_time()
    f1_time_list.append(t2 - t1)
    
    t1 = time.process_time()
    for i in range(iter_num):
        result = func(f2)
    t2 = time.process_time()
    f2_time_list.append(t2 - t1)
    
    t1 = time.process_time()
    for i in range(iter_num):
        result = func(f3)
    t2 = time.process_time()
    f3_time_list.append(t2 - t1)
    
    t1 = time.process_time()
    for i in range(iter_num):
        result = func(f4)
    t2 = time.process_time()
    f4_time_list.append(t2 - t1)
    
t1 = time.process_time()
for i in range(iter_num):
    result = my.Knapsack_Algorithm(f1, USE_ZASSENHAUS=False, output=False)
t2 = time.process_time()
f1_time_list.append(t2 - t1)
t1 = time.process_time()
for i in range(iter_num):
    result = my.Knapsack_Algorithm(f2, USE_ZASSENHAUS=False, output=False)
t2 = time.process_time()
f2_time_list.append(t2 - t1)
t1 = time.process_time()
for i in range(iter_num):
    result = my.Knapsack_Algorithm(f3, USE_ZASSENHAUS=False, output=False)
t2 = time.process_time()
f3_time_list.append(t2 - t1)

In [ ]:
print('f1:', f1)
print('f2:', f2)
print('f3:', f3)
print('f4:', f4)

In [44]:
latex(f3.as_expr())

'108 x^{20} - 522 x^{19} + 312 x^{18} + 1378 x^{17} - 1466 x^{16} - 711 x^{15} + 972 x^{14} - 255 x^{13} + 493 x^{12} + 863 x^{11} - 197 x^{10} - 1974 x^{9} - 359 x^{8} + 551 x^{7} - 28 x^{6} + 1122 x^{5} + 351 x^{4} - 540 x^{3} - 162 x^{2}'

In [ ]:
j = 100
t1 = time.process_time()
for i in range(j):
    factor(f1)
t2 = time.process_time()
print((t2 - t1))

In [ ]:
t1 = time.process_time()
factor(f2)
t2 = time.process_time()
print(t2 - t1)

In [15]:
t1 = time.process_time()
factor(f3)
t2 = time.process_time()
print(t2 - t1)

0.03125


In [ ]:
t1 = time.process_time()
factor(f4)
t2 = time.process_time()
print(t2 - t1)

In [45]:
t1 = time.process_time()
my.Knapsack_Algorithm(f1, USE_ZASSENHAUS=True, output=True)
t2 = time.process_time()
print(t2 - t1)

0.390625


In [38]:
t1 = time.process_time()
my._Zassenhaus_Algorithm(f3)
t2 = time.process_time()
print(t2 - t1)

g_list: [Poly(x**3 - 15*x**2 - x - 16, x, modulus=47), Poly(x + 23, x, modulus=47), Poly(x + 13, x, modulus=47), Poly(x - 3, x, modulus=47), Poly(x - 15, x, modulus=47), Poly(x**2 + 4*x - 1, x, modulus=47), Poly(x**4 + 23*x**3 - 12*x**2 - 5*x - 3, x, modulus=47), Poly(x - 14, x, modulus=47), Poly(x**3 + 16*x**2 - 15*x - 15, x, modulus=47), Poly(x + 18, x, modulus=47)]
h_list: [Poly(x**3 - 168874373487*x**2 - x - 168874373488, x, domain='ZZ'), Poly(x - 203082832910, x, domain='ZZ'), Poly(x - 110048909319, x, domain='ZZ'), Poly(x - 3, x, domain='ZZ'), Poly(x - 24870082821, x, domain='ZZ'), Poly(x**2 - 112527953703*x - 58668074160, x, domain='ZZ'), Poly(x**4 + 203082832909*x**3 - 211081719000*x**2 + 145944451789*x - 126638155530, x, domain='ZZ'), Poly(x - 115913523709, x, domain='ZZ'), Poly(x**3 + 168874373488*x**2 - 168874373487*x - 168874373487, x, domain='ZZ'), Poly(x - 58825464169, x, domain='ZZ')]
g_list: [Poly(x, x, modulus=7)]
h_list: [Poly(x, x, domain='ZZ')]
2.453125


In [29]:
t1 = time.process_time()
my.Zassenhaus_Algorithm(f2)
t2 = time.process_time()
print(t2 - t1)

0.625


In [ ]:
t1 = time.process_time()
my._Zassenhaus_Algorithm(f4)
t2 = time.process_time()
print(t2 - t1)

In [ ]:
%%time
factor(f1)

In [ ]:
%%time
my.Knapsack_Algorithm(f1, USE_ZASSENHAUS=False, output=True)

# SDP

In [ ]:
%%time
x = Symbol('x')
sdp = [1]
n_max = 30
for i in range(1, n_max+1):
    sdp.append(swinnerton_dyer_poly(i, x=x, polys=True).as_poly(domain=ZZ))
with open('SDP_list.pickle', 'wb') as file:
    pickle.dump(sdp, file)

In [ ]:
%%time
a = swinnerton_dyer_poly(10, x=x, polys=False).as_poly(domain=ZZ)

In [ ]:
with open('SDP_list.pickle', 'wb') as file:
    pickle.dump(sdp, file)

In [ ]:
from sympy import *
import numpy as np
import math
import tools as my
import random
import itertools
import time
import pickle

In [ ]:
with open('SDP_list.pickle', 'rb') as sdp:
    sdp = pickle.load(sdp)

In [ ]:
len(sdp)

In [ ]:
k = 5

In [ ]:
sdp[k]

In [ ]:
%%time
factor(sdp[k])

In [ ]:
%%time
my.Zassenhaus_Algorithm(sdp[k])

In [ ]:
%%time
my.Knapsack_Algorithm(sdp[k], USE_ZASSENHAUS=True, output=True)

In [ ]:
sdp[5]

In [ ]:
# L3: f2-> 13min 24sec

In [ ]:
# L3: f3-> 7h 30min 5sec

In [ ]:
# Zassenhaus: f4-> 47.5s

In [ ]:
# Knapsack: f4-> 8min 43s

In [ ]:
%%time
my.L3_Algorithm(f)

In [ ]:
from sympy import *
import numpy as np
import math
import tools as my
import random
import itertools
import time

In [ ]:
f2

In [ ]:
f3

In [ ]:
x = Symbol('x')
f1 = Poly(6*x**7 - 17*x**6 + 16*x**5 - 7*x**4 - 18*x**3 + 12*x**2 + 8*x, x, domain='ZZ')

In [ ]:
%%time
factor(f3)

In [ ]:
%%time
my.Knappsack_Algorithm(f2)

In [ ]:
%%time
my._Zassenhaus_Algorithm(f1)

In [ ]:
%%time
my.L3_Algorithm(f2)

In [ ]:
from sympy import *
import numpy as np
import math
import tools as my
import random
import itertools
import time

In [ ]:
x = Symbol('x')
h = Poly(3*x**5 + 18*x**3 - 2*x**2 - 12, x, domain=ZZ)
f = Poly(x**3 + 2*x**2 - 5*x - 6, x, domain=ZZ)

In [ ]:
%%time
my.Knappsack_Algorithm(h)

In [ ]:
my._Zassenhaus_Algorithm(f)

In [ ]:
f3

In [ ]:
list(reversed([1, 2]))[0]

In [ ]:
for i in range(1, 2):
    print(i)

In [ ]:
a = [Poly(840*x**11 + 2119*x**10 - 11753*x**9 + 16461*x**8 - 17166*x**7 + 11161*x**6 + 2520*x**5 - 5181*x**4 + 5618*x**3 - 3932*x**2 - 315*x + 588, x, domain='ZZ'), Poly(x, x, domain='ZZ')]

In [ ]:
for _ in a:
    print(my._Zassenhaus_Algorithm(_))

In [ ]:
f = Poly(1, x, domain=GF(5))

In [ ]:
f == f.zero

In [ ]:
degree(f)

In [ ]:
_f1

In [ ]:
_f2

In [ ]:
_f3

In [ ]:
factor(_f1)

In [ ]:
factor(_f2)

In [ ]:
factor(_f3)

In [ ]:
factor(f2)

In [ ]:
%%time
my.SquarefreeDecomposition(f1)

In [ ]:
%%time
my.SquarefreeDecomposition(f2)

In [ ]:
%%time
my.SquarefreeDecomposition(f3)

In [ ]:
my.

# memo

In [ ]:
%%time
_ = [my.my_prime(i) for i in range(10, 100, 3)]

In [ ]:
from sympy import *
import numpy as np
import math
import tools as my
import random
import itertools
import time

In [ ]:
if __name__ == '__main__':
    '''
    L3とナップザックの動作確認
    
    '''
    x = Symbol('x')
    deg = 50; coeffs_inf = -3; coeffs_sup = 3
    factors_num = 5
    
    '''
    while True:
        f = Poly(1, x, domain=ZZ)
        for i in range(factors_num):
            _deg = random.randint(2, deg//factors_num)
            _f = random_poly(x, _deg, coeffs_inf, coeffs_sup, domain=ZZ, polys=True)
            f *= _f
        f *= random_poly(x, deg - degree(f), coeffs_inf, coeffs_sup, domain=ZZ, polys=True)
        f = primitive(f)[1]
        if gcd(f, diff(f, x)) == 1:
            break
    '''
    f = Poly(216*x**15 + 468*x**14 + 480*x**13 
             + 8*x**12 - 628*x**11 - 1069*x**10 - 575*x**9 
             + 512*x**8 + 1053*x**7 + 367*x**6 - 236*x**5 - 464*x**4 
             - 268*x**3 + 316*x**2 + 48*x - 48, x, domain='ZZ')
    #'''
    print(f); print('\n')
    print(factor(f))
    print('------------------')
    
    #print('aaa' + str(my.L3_Algorithm(f)))
    #print('bbb' + str(my.Knappsack_Algorithm(f)))

In [ ]:
%%time
factor(f)

In [ ]:
%%time
my._Zassenhaus_Algorithm(f)

In [ ]:
%%time
my.Zassenhaus_Algorithm(f)

In [ ]:
%%time
my.Knappsack_Algorithm(f)

In [ ]:
%%time
my.L3_Algorithm(f)

In [ ]:
x = Symbol('x')
_f1 = my_random_poly_ZZ(2, 6, 5, 2); # deg = 15
_f2 = my_random_poly_ZZ(5, 2, 1, 4, 7, 2, 3, 6); # deg = 30
_f3 = my_random_poly_ZZ(5, 2, 7, 6, 11, 10, 6, 3); # deg = 50
#_f4 = my_random_poly_ZZ(7, 13, 21, 5, 4, 5, 12, 13, 17, 6, 7); # deg = 100